In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
os.listdir(FOLDER)

['mulliken_charges.csv',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'train.csv',
 'test.csv',
 'dipole_moments.csv',
 'potential_energy.csv',
 'structures.csv',
 'out',
 'scalar_coupling_contributions.csv',
 'dsgdb9nsd_000001.xyz',
 'structures.zip']

In [3]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
df_train = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [ ]:
def assign_atoms_index(df, molecule):
    se_0 = df.query('molecule_name == "{}"'.format(molecule))['atom_index_0']
    se_1 = df.query('molecule_name == "{}"'.format(molecule))['atom_index_1']
    se = pd.concat([se_0, se_1])
    return se.unique()

In [ ]:
def gen_dist_matrix(df_structure, mole_name, assigned_atoms_idx, num):
    atoms = ['H', 'C', 'N', 'O', 'F']
    dist_matrix_list = []
    for atom_idx0 in assigned_atoms_idx:
        loc0 = df_structures.query('molecule_name == "{}" and atom_index == {}'.format(mole_name, atom_idx0))[['x','y','z']].values.reshape(-1)
        atom_name0 = df_structures.query('molecule_name == "{}" and atom_index == {}'.format(mole_name, atom_idx0))['atom'].values[0] 
        
        atom_mole = df_structures.query('molecule_name == "{}"'.format(mole_name))['atom_index'].values
        mask = atom_mole != atom_idx0
        atom_idx_excl0 = atom_mole[mask]

        dist_list = []
        atom_list = []
        for i in atom_idx_excl0:
            loc1 = df_structures.query('molecule_name == "{}" and atom_index == {}'.format(mole_name, i))[['x','y','z']].values.reshape(-1)
            atom_name1 = df_structures.query('molecule_name == "{}" and atom_index == {}'.format(mole_name, i))['atom'].values[0]
            dist_list.append(1.0 / sum((loc0 - loc1)**2))
            atom_list.append(atom_name1)

        dist_arr = np.array(dist_list)
        atom_arr = np.array(atom_list)

        dist_arr_sorted = -np.sort(-dist_arr)
        atom_arr_sorted = atom_arr[np.argsort(-dist_arr)]

        dist_matrix = np.zeros([len(atoms), num])
        for j, atom in enumerate(atoms):
            len_sorted = len(dist_arr_sorted[atom_arr_sorted == atom])
            if num < len_sorted:
                dist_matrix[j, :] = dist_arr_sorted[atom_arr_sorted == atom][:num]
            else:
                dist_matrix[j, :len_sorted] = dist_arr_sorted[atom_arr_sorted == atom]

        dist_matrix_list.append(dist_matrix.reshape(-1))
    return np.array(dist_matrix_list)

In [ ]:
num = 5
mole_names = list(df_train['molecule_name'].value_counts().index)[:200]
dist_mat = np.zeros([0, num*5])
atoms_idx = np.zeros([0], dtype=np.int32)
molecule_names = np.empty([0])

start = time.time()

for name in mole_names:
    assigned_atoms_idx = assign_atoms_index(df_train, name)
    dist_mat_mole = gen_dist_matrix(df_structures, name, assigned_atoms_idx, num)
    name_arr = [name] * len(assigned_atoms_idx) 
    
    molecule_names = np.hstack([molecule_names, name_arr])
    atoms_idx = np.hstack([atoms_idx, assigned_atoms_idx])
    dist_mat = np.vstack([dist_mat, dist_mat_mole])
    
col_name_list = []
atoms = ['H', 'C', 'N', 'O', 'F']
for a in atoms:
    for n in range(num):
        col_name_list.append('dist_{}_{}'.format(a, n))
        
se_mole = pd.Series(molecule_names, name='molecule_name')
se_atom_idx = pd.Series(atoms_idx, name='atom_index')
df_dist = pd.DataFrame(dist_mat, columns=col_name_list)
df_distance = pd.concat([se_mole, se_atom_idx,df_dist], axis=1)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
df_distance.to_csv(OUTPUT + 'dictance1.csv', index=False)